***GENERATED CODE FOR gettingdata PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Date', 'transformation_label': 'String Indexer'}], 'feature': 'Date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '203', 'mean': '', 'stddev': '', 'min': '1/1/2022', 'max': '9/8/2019', 'missing': '0', 'distinct': '181'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Date'}, {'feature_label': 'Date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Country ', 'transformation_label': 'String Indexer'}], 'feature': ' Country ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' Australia ', 'max': ' Ukraine ', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Country '}, {'feature_label': ' Country ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Country ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' State ', 'transformation_label': 'String Indexer'}], 'feature': ' State ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' AZ ', 'max': 'UT', 'missing': '0', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' State '}, {'feature_label': ' State ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' State ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Description ', 'transformation_label': 'String Indexer'}], 'feature': ' Description ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' 3 car collision near intersection ', 'max': ' Wrong way driver strikes Tesla, Tesla burns ', 'missing': '0', 'distinct': '184'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Description '}, {'feature_label': ' Description ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Description ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Tesla driver ', 'transformation_label': 'String Indexer'}], 'feature': ' Tesla driver ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.0', 'stddev': '0.0', 'min': ' - ', 'max': '1', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Tesla driver '}, {'feature_label': ' Tesla driver ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Tesla driver ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Tesla occupant ', 'transformation_label': 'String Indexer'}], 'feature': ' Tesla occupant ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.5', 'stddev': '0.85', 'min': ' - ', 'max': '3', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Tesla occupant '}, {'feature_label': ' Tesla occupant ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Tesla occupant ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Other vehicle ', 'transformation_label': 'String Indexer'}], 'feature': ' Other vehicle ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.27', 'stddev': '0.74', 'min': ' - ', 'max': '4', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Other vehicle '}, {'feature_label': ' Other vehicle ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Other vehicle ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Cyclists/ Peds ', 'transformation_label': 'String Indexer'}], 'feature': ' Cyclists/ Peds ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.13', 'stddev': '0.34', 'min': ' - ', 'max': '2', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Cyclists/ Peds '}, {'feature_label': ' Cyclists/ Peds ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Cyclists/ Peds ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' TSLA+cycl / peds ', 'transformation_label': 'String Indexer'}], 'feature': ' TSLA+cycl / peds ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.2', 'stddev': '0.54', 'min': ' - ', 'max': '4', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' TSLA+cycl / peds '}, {'feature_label': ' TSLA+cycl / peds ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' TSLA+cycl / peds ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Model ', 'transformation_label': 'String Indexer'}], 'feature': ' Model ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '3.0', 'stddev': '0.0', 'min': ' - ', 'max': '3', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Model '}, {'feature_label': ' Model ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Model ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' AutoPilot claimed ', 'transformation_label': 'String Indexer'}], 'feature': ' AutoPilot claimed ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.17', 'stddev': '0.38', 'min': ' - ', 'max': '2', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' AutoPilot claimed '}, {'feature_label': ' AutoPilot claimed ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' AutoPilot claimed ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Verified Tesla Autopilot Death ', 'transformation_label': 'String Indexer'}], 'feature': ' Verified Tesla Autopilot Death ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '1.1', 'stddev': '0.32', 'min': ' - ', 'max': '2', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Verified Tesla Autopilot...'}, {'feature_label': ' Verified Tesla Autopilot Death ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            ' Verified Tesla Autopilot Death ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Source ', 'transformation_label': 'String Indexer'}], 'feature': ' Source ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' http://firststateupdate.com/2019/08/woman-injured-when-tesla-crashed-into-bay-to-the-beach-has-succumbed-to-her-injuries/ ', 'max': ' https://yle.fi/uutiset/3-12576787 ', 'missing': '0', 'distinct': '203'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Source '}, {'feature_label': ' Source ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Source ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Note ', 'transformation_label': 'String Indexer'}], 'feature': ' Note ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '203', 'mean': '', 'stddev': '', 'min': ' AP claimed in NHTSA complaint ',
                                                                                                                                                                                                                                                              'max': ' Contacted by close friend who disclosed the car was a Tesla. Awaiting further confirmation. ', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Note '}, {'feature_label': ' Note ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Note ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 1 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 1 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' Ariyanna Parsad ', 'max': ' Zhang ', 'missing': '0', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 1 '}, {'feature_label': ' Deceased 1 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 1 ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 2 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 2 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' Alex Francisco Cuesta ', 'max': ' Sarina Astorga ', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 2 '}, {'feature_label': ' Deceased 2 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 2 ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 3 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 3 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': ' Andrew James Chaves ', 'max': ' Quinn Cochlin ', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 3 '}, {'feature_label': ' Deceased 3 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 3 ')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': ' Deceased 4 ', 'transformation_label': 'String Indexer'}], 'feature': ' Deceased 4 ', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '203', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': ' Deceased 4 '}, {'feature_label': ' Deceased 4 ', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(' Deceased 4 ')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import datetime
import requests
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class NumtraConnector:

    def put(inStages, inStagesData, stageId, spark, config):
        path = eval(config)['server_url']
        baseType = eval(config)['baseType']
        results_url = eval(config)['results_url']
        server = eval(config)['server']
        originalfile = eval(config)['orignalKey']
        eval(config)['pathOnly']
        filename = eval(config)['filename']
        eval(config)['ser']
        eval(config)['user']
        eval(config)['password']
        eval(config)['authSource']
        eval(config)['user_id']
        eval(config)['parent_id']
        eval(config)['project_id']
        time = str(int(datetime.datetime.now().timestamp()))

        inStagesData[inStages[0]]

        print(path)
        print(baseType)
        print(results_url)
        print(server)
        print(originalfile)
        print(filename)

        args = {
            'url': path,
            'baseType': baseType,
            'originalfile': originalfile,
            'filename': time + filename
        }

        response = requests.post(results_url, args)
        return response


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run gettingdataHooks.ipynb
try:
	#sourcePreExecutionHook()

	tesladeathsdeaths = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Tesla Deaths - Deaths (3).csv', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'delimiter': ',', 'is_header': 'Use Header Line', 'filename': 'Tesla Deaths - Deaths (3).csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run gettingdataHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofedata = TransformationMain.run(tesladeathsdeaths,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Case #", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "203", "mean": "153.0", "stddev": "58.75", "min": "52", "max": "254", "missing": "0"}, "updatedLabel": "Case #"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "203", "mean": "2020.57", "stddev": "1.17", "min": "2018", "max": "2022", "missing": "0"}, "updatedLabel": "Year"}, {"transformationsData": [{"feature_label": "Date", "transformation_label": "String Indexer"}], "feature": "Date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": "1/1/2022", "max": "9/8/2019", "missing": "0", "distinct": "181"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Date"}, {"transformationsData": [{"feature_label": " Country ", "transformation_label": "String Indexer"}], "feature": " Country ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " Australia ", "max": " Ukraine ", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Country "}, {"transformationsData": [{"feature_label": " State ", "transformation_label": "String Indexer"}], "feature": " State ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " AZ ", "max": "UT", "missing": "0", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " State "}, {"transformationsData": [{"feature_label": " Description ", "transformation_label": "String Indexer"}], "feature": " Description ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " 3 car collision near intersection ", "max": " Wrong way driver strikes Tesla, Tesla burns ", "missing": "0", "distinct": "184"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Description "}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": " Deaths ", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "203", "mean": "1.21", "stddev": "0.52", "min": "1", "max": "4", "missing": "0"}, "updatedLabel": " Deaths "}, {"transformationsData": [{"feature_label": " Tesla driver ", "transformation_label": "String Indexer"}], "feature": " Tesla driver ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.0", "stddev": "0.0", "min": " - ", "max": "1", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Tesla driver "}, {"transformationsData": [{"feature_label": " Tesla occupant ", "transformation_label": "String Indexer"}], "feature": " Tesla occupant ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.5", "stddev": "0.85", "min": " - ", "max": "3", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Tesla occupant "}, {"transformationsData": [{"feature_label": " Other vehicle ", "transformation_label": "String Indexer"}], "feature": " Other vehicle ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.27", "stddev": "0.74", "min": " - ", "max": "4", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Other vehicle "}, {"transformationsData": [{"feature_label": " Cyclists/ Peds ", "transformation_label": "String Indexer"}], "feature": " Cyclists/ Peds ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.13", "stddev": "0.34", "min": " - ", "max": "2", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Cyclists/ Peds "}, {"transformationsData": [{"feature_label": " TSLA+cycl / peds ", "transformation_label": "String Indexer"}], "feature": " TSLA+cycl / peds ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.2", "stddev": "0.54", "min": " - ", "max": "4", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " TSLA+cycl / peds "}, {"transformationsData": [{"feature_label": " Model ", "transformation_label": "String Indexer"}], "feature": " Model ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "3.0", "stddev": "0.0", "min": " - ", "max": "3", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Model "}, {"transformationsData": [{"feature_label": " AutoPilot claimed ", "transformation_label": "String Indexer"}], "feature": " AutoPilot claimed ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.17", "stddev": "0.38", "min": " - ", "max": "2", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " AutoPilot claimed "}, {"transformationsData": [{"feature_label": " Verified Tesla Autopilot Death ", "transformation_label": "String Indexer"}], "feature": " Verified Tesla Autopilot Death ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "1.1", "stddev": "0.32", "min": " - ", "max": "2", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Verified Tesla Autopilot..."}, {"transformationsData": [{"feature_label": " Source ", "transformation_label": "String Indexer"}], "feature": " Source ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " http://firststateupdate.com/2019/08/woman-injured-when-tesla-crashed-into-bay-to-the-beach-has-succumbed-to-her-injuries/ ", "max": " https://yle.fi/uutiset/3-12576787 ", "missing": "0", "distinct": "203"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Source "}, {"transformationsData": [{"feature_label": " Note ", "transformation_label": "String Indexer"}], "feature": " Note ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " AP claimed in NHTSA complaint ", "max": " Contacted by close friend who disclosed the car was a Tesla. Awaiting further confirmation. ", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Note "}, {"transformationsData": [{"feature_label": " Deceased 1 ", "transformation_label": "String Indexer"}], "feature": " Deceased 1 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " Ariyanna Parsad ", "max": " Zhang ", "missing": "0", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 1 "}, {"transformationsData": [{"feature_label": " Deceased 2 ", "transformation_label": "String Indexer"}], "feature": " Deceased 2 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " Alex Francisco Cuesta ", "max": " Sarina Astorga ", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 2 "}, {"transformationsData": [{"feature_label": " Deceased 3 ", "transformation_label": "String Indexer"}], "feature": " Deceased 3 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": " Andrew James Chaves ", "max": " Quinn Cochlin ", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 3 "}, {"transformationsData": [{"feature_label": " Deceased 4 ", "transformation_label": "String Indexer"}], "feature": " Deceased 4 ", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "203", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": " Deceased 4 "}]}))

	#transformationPostExecutionHook(autofedata)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run gettingdataHooks.ipynb
try:
	#sinkPreExecutionHook()

	teslaaccidentsautofedata = NumtraConnector.fetch(spark, "{'samplefile': '/FileStore/platform/sampleData/65c6bc366b2c3a48ceeb9e19/part-00000-64749dd9-5ab7-4539-8b7d-a9f147354a41-c000.csv', 'samplecount': 254, 'originalcount': 254, 'orignalKey': '/FileStore/platform/uploadedSourceFiles/Tesla Deaths - Deaths (3).csv', 'pathOnly': '/Tesla Models Accidents Impact Prediction', 'project_id': '65c2eb84ef8cf2dbe1b4e12b', 'parent_id': '65c2eb84ef8cf2dbe1b4e12b', 'original_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6650f', 'field': 'Case #', 'alias': 'Case #', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66510', 'field': 'Year', 'alias': 'Year', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66511', 'field': ' Deaths ', 'alias': ' Deaths ', 'type': 'numeric', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66512', 'field': 'Date_stringindexer', 'alias': 'Date_stringindexer', 'type': 'real', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66513', 'field': ' Country _stringindexer', 'alias': ' Country _stringindexer', 'type': 'real', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66514', 'field': ' State _stringindexer', 'alias': ' State _stringindexer', 'type': 'real', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66515', 'field': ' Description _stringindexer', 'alias': ' Description _stringindexer', 'type': 'real', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66516', 'field': ' Tesla driver _stringindexer', 'alias': ' Tesla driver _stringindexer', 'type': 'numeric', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66517', 'field': ' Tesla occupant _stringindexer', 'alias': ' Tesla occupant _stringindexer', 'type': 'numeric', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66518', 'field': ' Other vehicle _stringindexer', 'alias': ' Other vehicle _stringindexer', 'type': 'real', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66519', 'field': ' Cyclists/ Peds _stringindexer', 'alias': ' Cyclists/ Peds _stringindexer', 'type': 'numeric', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6651a', 'field': ' TSLA+cycl / peds _stringindexer', 'alias': ' TSLA+cycl / peds _stringindexer', 'type': 'real', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6651b', 'field': ' Model _stringindexer', 'alias': ' Model _stringindexer', 'type': 'real', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6651c', 'field': ' AutoPilot claimed _stringindexer', 'alias': ' AutoPilot claimed _stringindexer', 'type': 'numeric', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6651d', 'field': ' Verified Tesla Autopilot Death _stringindexer', 'alias': ' Verified Tesla Autopilot Death _stringindexer', 'type': 'numeric', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6651e', 'field': ' Source _stringindexer', 'alias': ' Source _stringindexer', 'type': 'real', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b6651f', 'field': ' Note _stringindexer', 'alias': ' Note _stringindexer', 'type': 'real', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66520', 'field': ' Deceased 1 _stringindexer', 'alias': ' Deceased 1 _stringindexer', 'type': 'real', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66521', 'field': ' Deceased 2 _stringindexer', 'alias': ' Deceased 2 _stringindexer', 'type': 'real', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66522', 'field': ' Deceased 3 _stringindexer', 'alias': ' Deceased 3 _stringindexer', 'type': 'numeric', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c4c37211bad1c5b66523', 'field': ' Deceased 4 _stringindexer', 'alias': ' Deceased 4 _stringindexer', 'type': 'numeric', 'position': '20'}], 'actual_schema': [{'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6644f', 'field': 'Case #', 'alias': 'Case #', 'type': 'numeric', 'position': '0'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66450', 'field': 'Year', 'alias': 'Year', 'type': 'numeric', 'position': '1'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66451', 'field': 'Date', 'alias': 'Date', 'type': 'date', 'position': '2'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66452', 'field': ' Country ', 'alias': ' Country ', 'type': 'String', 'position': '3'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66453', 'field': ' State ', 'alias': ' State ', 'type': 'String', 'position': '4'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66454', 'field': ' Description ', 'alias': ' Description ', 'type': 'String', 'position': '5'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66455', 'field': ' Deaths ', 'alias': ' Deaths ', 'type': 'numeric', 'position': '6'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66456', 'field': ' Tesla driver ', 'alias': ' Tesla driver ', 'type': 'String', 'position': '7'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66457', 'field': ' Tesla occupant ', 'alias': ' Tesla occupant ', 'type': 'String', 'position': '8'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66458', 'field': ' Other vehicle ', 'alias': ' Other vehicle ', 'type': 'String', 'position': '9'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66459', 'field': ' Cyclists/ Peds ', 'alias': ' Cyclists/ Peds ', 'type': 'String', 'position': '10'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6645a', 'field': ' TSLA+cycl / peds ', 'alias': ' TSLA+cycl / peds ', 'type': 'String', 'position': '11'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6645b', 'field': ' Model ', 'alias': ' Model ', 'type': 'String', 'position': '12'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6645c', 'field': ' AutoPilot claimed ', 'alias': ' AutoPilot claimed ', 'type': 'String', 'position': '13'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6645d', 'field': ' Verified Tesla Autopilot Death ', 'alias': ' Verified Tesla Autopilot Death ', 'type': 'String', 'position': '14'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6645e', 'field': ' Source ', 'alias': ' Source ', 'type': 'String', 'position': '15'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b6645f', 'field': ' Note ', 'alias': ' Note ', 'type': 'String', 'position': '16'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66460', 'field': ' Deceased 1 ', 'alias': ' Deceased 1 ', 'type': 'String', 'position': '17'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66461', 'field': ' Deceased 2 ', 'alias': ' Deceased 2 ', 'type': 'String', 'position': '18'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66462', 'field': ' Deceased 3 ', 'alias': ' Deceased 3 ', 'type': 'String', 'position': '19'}, {'checked': True, 'inherited': True, 'is_categorical': False, 'bad_values': '', 'nullable': 'true', '_id': '65c6c3947211bad1c5b66463', 'field': ' Deceased 4 ', 'alias': ' Deceased 4 ', 'type': 'date', 'position': '20'}], 'server': 'https://ml.colaberry.com:443', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'delimiter': ',', 'file_type': 'Delimeted', 'filename': 'AutoFeaturedDataUpdated.csv', 'token': '', 'domain': '', 'is_header': 'Use Header Line', 'url': '/FileStore/platform/uploadedSourceFiles/part-00000-9770b2d6-3205-481f-8699-8d4146c45aaf-c000.csv', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
